In [79]:
from source.datasets.tf_datasets import HiveTFDataset
from source.models_task_specific.mb_token_classification import MusicBertAudioset


eval_dataset = HiveTFDataset("../datasets/audioset/audioset_v1_embeddings/eval/", num_workers=0)
train_dataset = HiveTFDataset("../datasets/audioset/audioset_v1_embeddings/bal_train/", num_workers=0)

model = MusicBertAudioset(527, RNN=False, num_encoder_layers=4, skip_vggish=True, multi_label=True, name="music_bert_audioset").cuda()

In [4]:
from source.utils.generic_utils import allDone

evals = 1

model.train_model(train_dataset.get_dataloader(batch_size=16),
                  eval_dataset.get_dataloader(batch_size=16),
                  epochs = 10, eval_per_epoch=evals)

allDone()

In [5]:
%matplotlib widget
from source.utils.plot_utils import plot_curve, smooth

loss = smooth(model.loss_curve.cpu().numpy(), 20)

plot_curve(loss, 1, color="red")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


In [6]:
import csv

with open("../datasets/audioset/class_labels_indices.csv") as f:
    
    x = csv.reader(f)
    labels_dict = {}
    
    for i, row in enumerate(x):
        
        if row[0] == 'index':
            continue
        
        labels_dict[int(row[0])] = row[2]
        
len(labels_dict)

527

In [6]:
from source.datasets.fast_datasets import *
# from source.models_task_specific.mb_classification import MusicBertClassifier


model = MusicBertAudioset(527, num_encoder_layers=4, skip_vggish=True, multi_label=True, name="audioset").cuda()

params = torch.load("models/audioset_classifier.pth")
del params['loss_curve']
del params['validation_curve']
                    
# model.load_state_dict(params, strict=False)
genre_dataset = GTZANFastDataset()

In [64]:
from source.datasets.sound_transforms import log_mel_spectrogram
import librosa

x_song, sr = librosa.load("../subtasks/GTZAN/hiphop/hiphop.00000.wav")

sample = log_mel_spectrogram(x_song, sr)

import IPython.display as ipd
ipd.Audio(x_song, rate=sr)

In [65]:
import numpy as np
import torch

model.BERT.skip_vggish = False

# song_id = np.random.randint(len(genre_dataset))
# sample = genre_dataset[song_id]
# sample_sound = torch.tensor(sample['song_features']).to(model.get_device()).unsqueeze(0)

sample_sound = torch.tensor(sample).to(model.get_device()).unsqueeze(0)

y = model(sample_sound).squeeze()
y.shape

torch.Size([31, 527])

In [66]:
cl = torch.argmax(y[0]).cpu()

#labels_dict[int(cl)]
data = []

for row in y:
    indices = torch.where(row > 0.02)[0].cpu().numpy()
    time_labels = [labels_dict[idx] for idx in indices]
    data.append(time_labels)

In [67]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

colors = ["red", "orange", "yellow", 'green', "blue", "purple"]


labels = set()
for sec in data:
    labels.update(sec)

removable = set()
for label in labels:
    bits = [label in row for row in data]
    
    if np.sum(bits) < 3:
        removable.update([label])

for l in removable:
    labels.remove(l)

fig = plt.figure(figsize=(12, 0.4*len(labels)))
ax = fig.add_subplot(111)
    
for label_id, label in enumerate(labels):
    bits = [label in row for row in data]
    
    for i, b in enumerate(bits):
        if b:
            ax.barh(label_id,
                 1,
                 left=i-0.5,
                 height=0.6,
                 align='center',
                 color=colors[label_id%6],
                 alpha = 0.8)

ax.set_xlim(xmin = -1, xmax = len(data))
ax.set_ylim(ymin = -1, ymax = len(labels))
plt.yticks(range(len(labels)), labels)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation


def update_line(num, line):
    i = num/10
    line.set_data( [i, i], [-1, 1])
    return line, 

def init():
    pass

fig = plt.figure()

x = np.arange(X_MIN, X_MAX, 0.1);
y = np.sin(x)

plt.scatter(x, y)

l , v = plt.plot(X_MIN, -1, X_MAX, 1, linewidth=2, color= 'red')

line_anim = animation.FuncAnimation(fig, update_line, X_MAX*10, init_func=init, fargs=(l, ), interval=100, blit=True)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
elem = ipd.Audio(x_song, rate=sr, autoplay=True)
elem

In [81]:
import multiprocessing as mp

mp.current_process()

<_MainProcess(MainProcess, started)>